## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,37.51,87,40,13.80,scattered clouds
1,1,Mazamari,PE,-11.3258,-74.5308,66.52,97,90,1.97,overcast clouds
2,2,Riyadh,SA,24.6877,46.7219,87.94,12,4,8.75,clear sky
3,3,Saint-Augustin,CA,51.2260,-58.6502,55.74,87,99,9.57,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,54.27,61,75,4.00,broken clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().count()

City_ID                253
City                   253
Country                253
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Riyadh,SA,87.94,clear sky,24.6877,46.7219,
8,Jaque,PA,79.25,moderate rain,7.5175,-78.1703,
10,Galle,LK,79.68,broken clouds,6.0367,80.2170,
19,Khandyga,RU,81.97,clear sky,62.6667,135.6000,
21,Avera,US,86.72,few clouds,33.1940,-82.5271,
23,Nabire,ID,86.41,scattered clouds,-3.3667,135.4833,
25,Itarema,BR,75.09,few clouds,-2.9248,-39.9167,
28,Port Moresby,PG,83.62,haze,-9.4431,147.1797,
29,Maine-Soroa,NE,75.52,overcast clouds,13.2114,12.0241,
32,Touros,BR,76.10,clear sky,-5.1989,-35.4608,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found --- skipping.")

Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.
Hotel not found --- skipping.


In [42]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Riyadh,SA,87.94,clear sky,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
8,Jaque,PA,79.25,moderate rain,7.5175,-78.1703,Imbu Bua Lodge
10,Galle,LK,79.68,broken clouds,6.0367,80.2170,Amangalla
19,Khandyga,RU,81.97,clear sky,62.6667,135.6000,Apel'sin
23,Nabire,ID,86.41,scattered clouds,-3.3667,135.4833,Hotel Nusantara Nabire
...,...,...,...,...,...,...,...
705,Pandan,PH,87.22,overcast clouds,14.0453,124.1694,Bonifacio Lodge
706,Yingkou,CN,77.92,light rain,40.6648,122.2283,Yingkou Juyin Junyue Hotel
711,Chumphon,TH,83.39,overcast clouds,10.3333,98.8333,สกาน จ๋า.
713,Dawei,MM,80.67,heavy intensity rain,14.0833,98.2000,Hotel Dawei


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl> 
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))